## Problema Das Minas de Carvão

**Problema Das Minas de Carvão -** Uma empresa planeja instalar uma usina termoelétrica e a maior dificuldade está em atender às exigências impostas pelas leis de proteção ambiental. Uma delas refere-se aos poluentes emitidos na atmosfera. O carvão necessário para aquecer as caldeiras é fornecido por três minas. As propriedades dos diferentes tipos de carvão produzidos em cada uma das minas estão indicadas na tabela abaixo. Os valores são relativos à queima de uma tonelada de carvão.

    Mina	Enxofre (em ppm)	Poeira (em Kg)	Vapor produzido (em Kg)
    M1          1.100               1,7                  24.000
    M2          3.500               3,2                  36.000
    M3          1.300               2,4                  28.000

Os 03 tipos de carvão podem ser misturados e combinados em qualquer proporção. As emissões de poluentes e de vapor de uma mistura qualquer são proporcionais aos valores indicados na tabela. As exigências ambientais requerem que:

Para cada tonelada de carvão queimada a quantidade de enxofre não deve ser superior a 2.500 ppm;

Para cada tonelada de carvão queimada a quantidade de poeira não deve ser superior a 2,8 kg.

Os engenheiros querem determinar qual é a quantidade máxima de vapor (energia) que é possível gerar com a queima de uma tonelada de carvão.

x_j = quantidade em tonelada de carvão da Mina Mj para j = 1,2,3

max 24000 * x_1 + 36000 * x_2 + 28000 * x_3

sujeito a: 

1100 * x_1 + 3500 * x_2 + 1300 * x_3 <= 2500

1,7 * x_1 + 3,2 * x_2 + 2,4 * x_3 <= 2,8

x_1 + x_2 + x_3 = 1

x_1, x_2, x_3 >= 0

In [1]:
from docplex.mp.model import Model
import cplex

m = Model(name='Minas de Carvão')

x = dict()

for i in range(1,4):
    x[i] = m.continuous_var(name='x_{0}'.format(i))

m.add_constraint(1100*x[1] + 3500*x[2] + 1300*x[3] <= 2500)
m.add_constraint(1.7*x[1] + 3.2*x[2] + 2.4*x[3] <= 2.8)
m.add_constraint(x[1] + x[2] + x[3] == 1)

for i in range(1,4):
    m.add_constraint(x[i]>=0)

m.maximize(24000 * x[1] + 36000 * x[2] + 28000 * x[3])
m.solve()
print(m.solution)


solution for: Minas de Carvão
objective: 32173.9
x_1=0.058
x_2=0.551
x_3=0.391



### Usando o método de força bruta

1) testar, para cada combinação de m colunas de A, se elas formam uma base; 

2) calcular, para cada base, a solução básica associada a ela; 

3) calcular, para todas as SBV obtidas, o valor da função objetivo.

x_j = quantidade em tonelada de carvão da Mina Mj para j = 1,2,3

max 24000 * x_1 + 36000 * x_2 + 28000 * x_3

sujeito a: 

1100 * x_1 + 3500 * x_2 + 1300 * x_3 + x_4 = 2500

1,7 * x_1 + 3,2 * x_2 + 2,4 * x_3 + x_5 = 2,8

x_1 + x_2 + x_3 = 1

x_1, x_2, x_3, x4, x5 >= 0

In [2]:
from math import factorial
from __future__ import print_function
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
A = np.array([[1100, 3500, 1300, 1, 0],
              [1.7, 3.2, 2.4, 0, 1],
              [1, 1, 1, 0, 0]])

In [4]:
b = np.array([[2500], 
              [2.8],
              [1]])

In [5]:
c = np.array([24000, 36000, 28000, 0, 0])

In [6]:
m = len(A)
print('m:', m)

n = len(A[0])
print('n:', n)

combinacoes = factorial(n)/(factorial(m)*(factorial(n-m)))

print('O número máximo de soluções básicas:', combinacoes)

m: 3
n: 5
O número máximo de soluções básicas: 10.0


In [7]:
def verifica_solucao_viavel(x):
    '''
        Função que verifica se a solução é viável
        
        >>> verifica_solucao_viavel([0, 0, [125.], 0, [125.], [475.], 0])
            Esta é uma solução básica viável
            True
        >> verifica_solucao_viavel([0, 0, 0, [300.], [-950.], 0, [500.]])
            Esta NÃO é uma solução básica viável
            False
        >> verifica_solucao_viavel([1, 0, 1, 0, 1, 0, 0])
            Esta é uma solução básica viável
            True
    '''
    
    xB_bool = []

    for w in range(0, len(x)):
        if(x[w] >= 0):
            xB_bool.append(True)           
        else:
            xB_bool.append(False)

        try:
            if(x[w][0] >= 0):
                xB_bool.append(True)
            else:
                xB_bool.append(False)
        except:
            pass
            
    if(all(xB_bool)):
        print('Esta é uma solução básica viável')
        return True
    else:
        print('Esta NÃO é uma solução básica viável')
        return False

In [8]:
solucoes_viaveis = np.array([[0,0,0], 0,0,0])
todas_solucoes = np.array([[0,0,0], 0,0,0])
cont_x = 0

for w in range(0, n):
    for y in range(w+1, n):
        for z in range(y+1, n):
            
            # Id da solução
            cont_x = cont_x + 1;
            print('\nx', cont_x)
            
            # Conjunto de índices IB
            IB = [w,y,z]
            print('IB =', IB[0]+1, IB[1]+1, IB[2]+1)
            
            # B
            B = np.array([A[0][w], A[0][y], A[0][z]])
            for v in range (1, m):
                B = np.row_stack(tup=(B, [A[v][w], A[v][y], A[v][z]]))
            print('B =', B, '\n')
            
            # Verificando se B é uma base
            v, V =  np.linalg.eig(B)
            if(v.all() == 0):
                print('A matriz B não é uma base')
                print('A linha LD é:')
                print (B[v == 0,:])
                print('\n---------------------------------------------------------------------------\n')
                
                # Colocar na tabela de soluções básicas, com informações vazias
                todas_solucoes = np.row_stack((todas_solucoes, [[IB[0]+1, IB[1]+1, IB[2]+1], cont_x, [], []]))
                continue

            # B-1
            try:
                B1 = np.linalg.inv(B)
                print('B-1 =', B1, '\n')
            except:
                print("B-1 não existe")
                todas_solucoes = np.row_stack((todas_solucoes, [[IB[0]+1, IB[1]+1, IB[2]+1], cont_x, [], []]))
                print('\n---------------------------------------------------------------------------\n')
                continue

            # xB
            xB = B1.dot(b)
            print('xB =', xB, '\n')
            
            # x
            x = []
            for v in range (0, n):              
                if(v == IB[0]):
                    x.append(np.round(xB[0], 3))
                elif(v == IB[1]):
                    x.append(np.round(xB[1], 3))
                elif(v == IB[2]):
                    x.append(np.round(xB[2], 3))
                else:
                    x.append(0)
            print('x =', x, '\n')
                
            # cTx
            cTx = np.transpose(c).dot(x)
            print('cTx =', cTx, '\n')
                
            # Colocar na tabela de soluções básicas
            todas_solucoes = np.row_stack((todas_solucoes, [[IB[0]+1, IB[1]+1, IB[2]+1], cont_x, cTx, x]))
            
            # Se a solução for viável, colocar na tabela de soluções viáveis
            if(verifica_solucao_viavel(x) == True):
                solucoes_viaveis = np.row_stack((solucoes_viaveis, [[IB[0]+1, IB[1]+1, IB[2]+1], cont_x, cTx, x]))
            
            print('\n---------------------------------------------------------------------------\n')



x 1
IB = 1 2 3
B = [[1.1e+03 3.5e+03 1.3e+03]
 [1.7e+00 3.2e+00 2.4e+00]
 [1.0e+00 1.0e+00 1.0e+00]] 

B-1 = [[ 5.79710145e-04 -1.59420290e+00  3.07246377e+00]
 [ 5.07246377e-04 -1.44927536e-01 -3.11594203e-01]
 [-1.08695652e-03  1.73913043e+00 -1.76086957e+00]] 

xB = [[0.05797101]
 [0.55072464]
 [0.39130435]] 

x = [array([0.058]), array([0.551]), array([0.391]), 0, 0] 

cTx = [32176.] 

Esta é uma solução básica viável

---------------------------------------------------------------------------


x 2
IB = 1 2 4
B = [[1.1e+03 3.5e+03 1.0e+00]
 [1.7e+00 3.2e+00 0.0e+00]
 [1.0e+00 1.0e+00 0.0e+00]] 

B-1 = [[ 0.00000000e+00 -6.66666667e-01  2.13333333e+00]
 [-0.00000000e+00  6.66666667e-01 -1.13333333e+00]
 [ 1.00000000e+00 -1.60000000e+03  1.62000000e+03]] 

xB = [[ 2.66666667e-01]
 [ 7.33333333e-01]
 [-3.60000000e+02]] 

x = [array([0.267]), array([0.733]), 0, array([-360.]), 0] 

cTx = [32796.] 

Esta NÃO é uma solução básica viável

------------------------------------------------

<ipython-input-8-85446df97053>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  solucoes_viaveis = np.array([[0,0,0], 0,0,0])
<ipython-input-8-85446df97053>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  todas_solucoes = np.array([[0,0,0], 0,0,0])
<ipython-input-8-85446df97053>:63: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cTx

In [9]:
## Formatando os valores do todas_solucoes para sair bonitinho na tabela

for x in range(1, len(todas_solucoes)):
    try:
        todas_solucoes[x][2] = todas_solucoes[x][2][0]
    except:
        pass

for x in range(1, len(todas_solucoes)):
    for y in range(0, len(todas_solucoes)):
        try:
            todas_solucoes[x][3][y] = todas_solucoes[x][3][y][0]
        except:
            pass

todas_solucoes

array([[[0, 0, 0], 0, 0, 0],
       [[1, 2, 3], 1, 32176.0, [0.058, 0.551, 0.391, 0, 0]],
       [[1, 2, 4], 2, 32796.0, [0.267, 0.733, 0, -360.0, 0]],
       [[1, 2, 5], 3, 30996.0, [0.417, 0.583, 0, 0, 0.225]],
       [[1, 3, 4], 4, 30284.0, [-0.571, 0, 1.571, 1085.714, 0]],
       [[1, 3, 5], 5, 52000.0, [-6.0, 0, 7.0, 0, -3.8]],
       [[1, 4, 5], 6, 24000.0, [1.0, 0, 0, 1400.0, 1.1]],
       [[2, 3, 4], 7, 32000.0, [0, 0.5, 0.5, 100.0, 0]],
       [[2, 3, 5], 8, 32360.0, [0, 0.545, 0.455, 0, -0.036]],
       [[2, 4, 5], 9, 36000.0, [0, 1.0, 0, -1000.0, -0.4]],
       [[3, 4, 5], 10, 28000.0, [0, 0, 1.0, 1200.0, 0.4]]], dtype=object)

In [10]:
print("Todas as soluções:")

todas_solucoes = pd.DataFrame(todas_solucoes[1:,:], columns=['Conjunto de índices base', 'Id da solução', 'cTx', 'Solução básica associada à base'])
todas_solucoes.sort_values(by=['Id da solução'])

todas_solucoes

Todas as soluções:


,Conjunto de índices base,Id da solução,cTx,Solução básica associada à base
0,"[1, 2, 3]",1,32176.0,"[0.058, 0.551, 0.391, 0, 0]"
1,"[1, 2, 4]",2,32796.0,"[0.267, 0.733, 0, -360.0, 0]"
2,"[1, 2, 5]",3,30996.0,"[0.417, 0.583, 0, 0, 0.225]"
3,"[1, 3, 4]",4,30284.0,"[-0.571, 0, 1.571, 1085.714, 0]"
4,"[1, 3, 5]",5,52000.0,"[-6.0, 0, 7.0, 0, -3.8]"
5,"[1, 4, 5]",6,24000.0,"[1.0, 0, 0, 1400.0, 1.1]"
6,"[2, 3, 4]",7,32000.0,"[0, 0.5, 0.5, 100.0, 0]"
7,"[2, 3, 5]",8,32360.0,"[0, 0.545, 0.455, 0, -0.036]"
8,"[2, 4, 5]",9,36000.0,"[0, 1.0, 0, -1000.0, -0.4]"
9,"[3, 4, 5]",10,28000.0,"[0, 0, 1.0, 1200.0, 0.4]"


In [11]:
qtd_solucoes_basicas = list(map(lambda z, sb=0: sb if z == [] else sb+1, todas_solucoes.loc[:, 'cTx'])).count(1)

print("\nQuantidade de soluções básicas: ", qtd_solucoes_basicas)


Quantidade de soluções básicas:  10


<ipython-input-11-243826ed9650>:1: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  qtd_solucoes_basicas = list(map(lambda z, sb=0: sb if z == [] else sb+1, todas_solucoes.loc[:, 'cTx'])).count(1)


In [12]:
## Formatando os valores do solucoes_viaveis para sair bonitinho na tabela

for x in range(1, len(solucoes_viaveis)):
    try:
        solucoes_viaveis[x][2] = solucoes_viaveis[x][2][0]
    except:
        pass

for x in range(1, len(solucoes_viaveis)):
    for y in range(0, len(solucoes_viaveis)):
        try:
            solucoes_viaveis[x][3][y] = solucoes_viaveis[x][3][y][0]
        except:
            pass

solucoes_viaveis

array([[[0, 0, 0], 0, 0, 0],
       [[1, 2, 3], 1, 32176.0, [0.058, 0.551, 0.391, 0, 0]],
       [[1, 2, 5], 3, 30996.0, [0.417, 0.583, 0, 0, 0.225]],
       [[1, 4, 5], 6, 24000.0, [1.0, 0, 0, 1400.0, 1.1]],
       [[2, 3, 4], 7, 32000.0, [0, 0.5, 0.5, 100.0, 0]],
       [[3, 4, 5], 10, 28000.0, [0, 0, 1.0, 1200.0, 0.4]]], dtype=object)

In [13]:
print("Soluções viáveis:")

solucoes_viaveis = pd.DataFrame(solucoes_viaveis[1:,:], columns=['Conjunto de índices base', 'Id da solução', 'cTx', 'Solução básica associada à base'])
solucoes_viaveis.sort_values(by=['Id da solução'])

solucoes_viaveis

Soluções viáveis:


,Conjunto de índices base,Id da solução,cTx,Solução básica associada à base
0,"[1, 2, 3]",1,32176.0,"[0.058, 0.551, 0.391, 0, 0]"
1,"[1, 2, 5]",3,30996.0,"[0.417, 0.583, 0, 0, 0.225]"
2,"[1, 4, 5]",6,24000.0,"[1.0, 0, 0, 1400.0, 1.1]"
3,"[2, 3, 4]",7,32000.0,"[0, 0.5, 0.5, 100.0, 0]"
4,"[3, 4, 5]",10,28000.0,"[0, 0, 1.0, 1200.0, 0.4]"


In [14]:
qtd_solucoes_viaveis = len(solucoes_viaveis)
print("\nQuantidade de soluções viáveis: ", qtd_solucoes_viaveis)


Quantidade de soluções viáveis:  5


In [15]:
## Digite 1 se você deseja maximizar ou 2 se você deseja minimizar
entrada = 1

if(entrada == 1):
    id_solucao_otima = solucoes_viaveis['cTx'].astype(float).argmax()
elif(entrada == 2):
    id_solucao_otima = solucoes_viaveis['cTx'].astype(float).argmin()
    
solucao_otima = solucoes_viaveis['cTx'][id_solucao_otima]
geram_solucoes_otimas =  solucoes_viaveis['cTx'][id_solucao_otima]
print('Solução Ótima:', solucao_otima)


Solução Ótima: 32176.0


In [16]:
print('\nSoluções que geram a solução ótima:\n')

geram_solucoes_otimas = solucoes_viaveis.loc[solucoes_viaveis['cTx'] == solucao_otima, :]
geram_solucoes_otimas


Soluções que geram a solução ótima:



,Conjunto de índices base,Id da solução,cTx,Solução básica associada à base
0,"[1, 2, 3]",1,32176.0,"[0.058, 0.551, 0.391, 0, 0]"


In [17]:
print("\nQuantidade de soluções viáveis: ", qtd_solucoes_viaveis)
print("\nQuantidade de soluções básicas: ", qtd_solucoes_basicas)
print('\nO número máximo de soluções básicas:', combinacoes)
print('\nSolução Ótima:', solucao_otima)


Quantidade de soluções viáveis:  5

Quantidade de soluções básicas:  10

O número máximo de soluções básicas: 10.0

Solução Ótima: 32176.0
